<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Простая реализация метода Эйлера

## Modules – Basics
<section class="post-meta">
By Niels Henrik Aase, Thorvald Ballestad, Eilif Sommer Øyre and Jon Andreas Støvneng
</section>
Last edited: November 5th 2019

___
Этот блокнот служит демонстрацией метода Эйлера, простейшего численного метода решения дифференциального уравнения первого порядка. Наше внимание будет сосредоточено на __реализации__, поскольку мы предоставим только самую важную теорию. Для более подробного описания метода Эйлера ознакомьтесь с нашим другим блокнотом на *[Euler's method](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/eulers_method.ipynb)*.
___

## Вступление

Во всех видах естественных наук мы сталкиваемся с дифференциальными уравнениями. Эти уравнения могут быть довольно сложными для аналитического решения, и в большинстве случаев аналитического решения не существует. 

Для многих студентов понимание того, как решать обыкновенные дифференциальные уравнения, сокращенно ОДУ, может быть трудоемким и разочаровывающим процессом. Цель этого блокнота - просто продемонстрировать, как можно решить ОДУ. Не будем фокусироваться на теории, лежащей в основе метода Эйлера, мы сосредоточимся исключительно на __реализации__ нашего решателя ОДУ. В идеале читатель должен также понимать теорию и возможные подводные камни использования метода Эйлера, поэтому мы настоятельно рекомендуем также прочитать наш другой модуль по *[методу Эйлера](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/eulers_method.ipynb)* (Примечание: Этот модуль будет обновлен вскоре после публикации этой записной книжки). 

С точки зрения программирования, уровень этого блокнота довольно прост, но от читателя ожидается знакомство с базовым Python и в меньшей степени с библиотекой NumPy. Для получения дополнительной информации о последнем, ознакомьтесь с нашим *[блокнотом по numpy](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/introduction_to_numpy.ipynb)*.

Наконец, мы хотели бы объяснить выбор того, какую физическую систему мы собираемся решить. Самые грубые модели для моделирования роста бактерий - отличный выбор для демонстрации метода Эйлера, так как дифференциальное уравнение, управляющее этим ростом, довольно простое.

## Теория

### Рост популяции

Прежде чем мы начнем программирование, мы дадим краткое введение в рост бактерий. Бактерии размножаются бинарным делением [[1]](https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BF%D1%80%D0%BE%D0%BA%D0%B0%D1%80%D0%B8%D0%BE%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D1%85_%D0%BA%D0%BB%D0%B5%D1%82%D0%BE%D0%BA). Это процесс, при котором одна бактериальная клетка расщепляется надвое, создавая две бактериальные клетки. Две новые бактериальные клетки могут снова разделиться на две, создавая таким образом четыре бактериальные клетки. На самом деле одна бактериальная клетка может разделиться надвое только определенное количество раз, но в нашем проекте мы моделируем скорость нашего роста так, как если бы это было не так.

Затем мы можем записать наше дифференциальное уравнение в виде

\begin{equation}
\frac{dy}{dt} = ky(t),
\label{ODE}\quad (1)
\end{equation}

где $y(t)$ - население $y$ в момент времени $t$, а $k$ - некоторая константа пропорциональности. Если мы используем дни в качестве единицы времени и предположим, что размер популяции при $ t= 1$ вдвое превышает размер популяции при $t=0$

\begin{equation}
y(1) = 2y(0)
\label{Initial},\quad (2)
\end{equation}

мы можем решить дифференциальное уравнение аналитически. Комбинируя уравнения (1) и (2), мы получаем

\begin{equation}
y(t) = \mathrm{e}^{kt} = \mathrm{e}^{\mathrm{ln}(2)t} = 2^{t}. 
\label{Ana_sol}\quad (3)
\end{equation}

Уравнение (3) полностью описывает размер популяции в момент времени $t$.

### Метод Эйлера

В этом параграфе мы попытаемся изложить минимум теории, касающейся метода Эйлера. Опять же, мы настоятельно рекомендуем прочитать наш полный модуль по *[методу Эйлера](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/eulers_method.ipynb)*, поскольку некоторые концепции могут быть представлены недостаточно глубоко. Однако этих пунктов должно быть достаточно для осуществления.

Прежде всего, при численном решении дифференциальных уравнений нам нужно __дискретизировать__ нашу переменную времени $t$. $t$ больше не является непрерывной переменной, вместо этого мы пишем

$$
t_n = t_0 + nh, \quad \mathrm{with} \quad n = 0, 1, 2,..., N,
$$

где $t_0$ - это значение времени, при котором мы знаем наш начальный размер популяции, а $h$ - размер между соседними дискретными значениями времени. Отношение между $N$ и $h$ задается

$$
h = \frac{t_N - t_0}{N},
$$

где $N$ - количество дискретных временных точек, в то время как $t_N$ и обозначают наибольшее значение времени.

В общем, наше численное приближение будет лучше, если мы выберем небольшой $h$. Обратите внимание, что по мере уменьшения размера $h$ увеличивается количество дискретных значений времени между $t_0$ и $t_N$. Мы платим за повышенную точность, увеличением количества необходимых вычислений, тем самым увеличивая время выполнения нашей программы.

Дифференциальное уравнение первого порядка без явной зависимости от времени может быть записано в наиболее общем виде

\begin{equation}
\frac{dy}{dt} = g(y),
\label{gen_Euler}
\end{equation}

где $g(y)$ - уравнение, определяющее скорость изменения нашей функции $y(t)$. В нашем случае $g(y) = ky$.

Если мы знаем размер популяции в $t_0$ и обозначаем это значение как $y_0$, мы можем использовать метод Эйлера, чтобы найти приближение размера популяции в $t_1 = t_0 +h$. Размер популяции в $t_1$ обозначается как $y_1$. Это приближение может быть рассчитано по формуле

$$
y_1 = y_0 + hg(y_0).
$$

Теперь, чтобы найти размер популяции $y_2$ при $t_2 = t_1 + h = t_0 + 2h$, мы используем ту же формулу, но с $y_1$ вместо $y_0$

$$
y_2 = y_1 + h g(y_1).
$$

Наиболее общая форма метода Эйлера записывается как

$$
y_{n+1} = y_n + h g(y_n),
\label{Euler}
$$
где у нас есть начальное условие, что

$$
y(t_0) = y_0.
$$

Теперь мы можем реализовать метод Эйлера в Python и сравнить наши численные результаты с аналитическим решением, приведенным в уравнении (3).

## Реализация

In [ ]:
# Импорт необходимых библиотек
import numpy as np # NumPy используется для генерации массивов и выполнения некоторых математических операций
import matplotlib.pyplot as plt # Используется для построения графиков результатов

# Обновление параметров рисунка
# Обратите внимание, что это не важно для любого кода, 
# это просто для установки соответствующих размеров рисунка и текста
newparams = {'figure.figsize': (15, 7), 'axes.grid': False,
             'lines.markersize': 10, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral', 'figure.dpi': 200}
plt.rcParams.update(newparams)

In [ ]:
def step_Euler(y, h, f):
    """Эта функция Python выполняет один шаг метода Эйлера.
    
    Параметры:
            y: Численное приближение y в момент времени t
            h: Размер шага
            f: RHS нашего ОДУ (RHS = правая сторона). 
            Может быть любой функцией, которая имеет только y в качестве переменной.
        Returns:
            next_y: Численное приближение y в момент времени t+h
    """
    next_y = y + h * f(y)
    return next_y


def full_Euler(h, f, y_0 = 1, start_t = 0, end_t = 1):
    """  Полная числовая аппроксимация ОДУ в заданном интервале времени. Выполняет последовательные шаги Эйлера
    с размером шага h от времени начала до времени окончания. Также учитываются начальные значения ОДУ
    
    Параметры:
            h: Размер шага
            f: RHS нашего ОДУ
            y_0 : Начальное условие для y при t = start_t
            start_t : Время в начальном состоянии, t_0
            end_t : Конец интервала, на котором выполняется метод Эйлера, t_N
        Returns:
            y_list: Численное приближение y в моменты времени t_list
            t_list: Равномерно распределенный дискретный список времени с интервалом h. 
                    Время начала = start_t, а время окончания = end_t 
    """
    # Количество шагов дискретизации
    N = int((end_t - start_t) / h)
    # Следуя обозначениям в теории, мы имеем N+1 дискретных значений времени, линейно разнесенных
    t_list = np.linspace(start_t, end_t, N + 1)
    
    # Инициализируйте массив для хранения значений y
    y_list = np.zeros(N + 1)
    # Назначить начальное условие первому элементу
    y_list[0] = y_0
    
    # Назначьте остальную часть массива с помощью N Euler_steps
    for i in range(0, N):
        y_list[i + 1] = step_Euler(y_list[i], h, f)
    # При N + 1 значениях времени мы можем вычислить только N значений y. Таким образом, 
    # мы не возвращаем последний элемент нашего списка времени. Мы достигаем этого, написав t_list[:-1]
    return y_list, t_list 

Теперь, когда мы определили наши функции, мы можем смоделировать популяцию бактерий. Нам нужно только определить нашу RHS (правую часть) нашего дифференциального уравнения, которое мы в теоретической части обозначили как $g(y)$.

In [ ]:
def g(y):
    """Определяет правую часть нашего дифференциального уравнения. В нашем случае роста бактерий g(y) = k*y
    
    Параметры:
            y: Численное приближение y в момент времени t
        Возвращается:
            growth_rate: Текущая численность населения, умноженная на константу пропорциональности. В этом случае она
    равна ln(2)
    """
    growth_rate = np.log(2)*y
    return growth_rate

# Теперь мы можем найти численные результаты метода Эйлера и сравнить их с аналитическим решением

# Входные параметры
y_0 = 1  # Начальный размер популяции, т. е. одна бактерия
h = 0.01 # Размер шага
t_0 = 0  # Мы определяем время при нашем первоначальном наблюдении как 0
t_N = 10 # через 10 дней после нашего первоначального наблюдения за одной бактерией


# Вычисление результатов по Эйлеру и их построение
y_list, t_list = full_Euler(h, g, y_0, t_0, t_N)
plt.plot(t_list, y_list, label="Numerical", linewidth=1)

# Построение аналитического решения, полученного ранее
plt.plot(t_list,np.power(2, t_list), label="Analytical", linewidth=1)

# Чтобы сюжет выглядел красиво
plt.legend()
plt.title("Размер популяции бактериальной колонии в зависимости от времени")
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$y$ [# bacteria]')
plt.show()

# Давайте посмотрим, насколько далека наша численная аппроксимация через 5 дней.

last_analytical = np.power(2,t_list[-1]) # Извлечение последнего элемента аналитического решения
last_numerical = y_list[-1] # Извлечение последнего элемента численного решения

print("Через 10 дней наше численное приближение бактерий отличается на: %.2f" %(last_analytical - last_numerical))

Мы видим, что наша модель довольно хорошо работает с размером шага $h=0,01$, поскольку она отклоняется от аналитического решения только на 24 бактериальных клетки, или на 2,4%. Выбрав меньший $h$, мы могли бы повысить точность нашего результата. Например, выбрав $h=0,001$, мы получим отклонение только в 2,4 бактериальных клетки, или 0,24 %.

Читатель может задаться вопросом, почему мы хотели бы использовать метод Эйлера в этом случае, поскольку у нас есть прекрасное аналитическое решение. Это дифференциальное уравнение имеет аналитическое решение, однако это не всегда так. Мы выбрали этот пример только потому, что можем сравнить наши численные результаты с правильным ответом и, таким образом, проверить нашу процедуру. Заменим наше начальное уравнение для роста (1), на то, что называется логистической моделью роста в форме

\begin{equation}
\frac{dy}{dt} = ky(1 - \frac{y}{m}),
\label{logistic}
\end{equation}

где $m$ - константа, мы не можем решить это уравнение аналитически. Однако мы все еще можем использовать метод Эйлера!

In [ ]:
def r(y):
    """Определяет правую часть нашего нового дифференциального уравнения. В нашем случае
роста бактерий h(y) = k*y*(1-y/m). В этом случае мы задаем k = 1, а m = 100.
    
    Parameters:
            y: Численное приближение y в момент времени t
        Returns:
            growth_rate: Текущий темп роста для населения с численностью населения = y
    """
    k = 1
    m = 100
    growth_rate = k * y * (1 - y / m)
    return growth_rate


# Входные параметры

y_0 = 1 # Начальная численность населения
h = 0.01 # Размер шага
t_0 = 0 # Мы определяем время при нашем первоначальном наблюдении как 0
t_N = 10 # через 10 дней после нашего первоначального наблюдения за одной бактерией


# Вычисление результатов по Эйлеру и их построение
y_list, t_list = full_Euler(h, r, y_0, t_0, t_N)
plt.plot(t_list, y_list)


# Чтобы сюжет выглядел красиво
plt.title("Размер популяции бактериальной колонии в зависимости от времени с использованием логистической модели роста")
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$y$ [# bacteria]')
plt.show()

Этот график прекрасно иллюстрирует суть метода Эйлера и численные решения ОДУ в целом. Даже если у нас нет аналитического решения для проверки наших результатов, мы все равно можем извлечь важную информацию из наших численных результатов и проверить результаты с помощью эмпирики. Константа $m$ в \eqref{logistic} называется пропускной способностью, и это название проявляется на нашем графике, поскольку мы наблюдаем, что популяция увеличивается экспоненциально в первые пять дней, но затем, кажется, сходится к 100 бактериям через 8 дней. Физически возможное объяснение может заключаться в том, что наша система располагает ресурсами только для поддержания популяции из 100 бактерий, и что лишние бактерии погибнут.


## Вывод
Несмотря на то, что метод Эйлера очень прост в реализации, он по-прежнему является мощным инструментом для решения ОДУ. В этом блокноте мы использовали метод Эйлера для решения дифференциального уравнения, управляющего ростом популяции в бактериальной колонии, но реализация также может быть использована для решения любого ОДУ без явной зависимости от времени. Однако, как мы уже упоминали, читатель должен быть знаком с различными подводными камнями и ограничениями метода Эйлера. Ознакомьтесь с нашей более подробной и полной записной книжкой по *[методу Эйлера](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/eulers_method.ipynb)* если вам интересно узнать больше!